## Prepare

In [6]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cd /content/drive/MyDrive/cs231n_final_project/

/content/drive/MyDrive/cs231n_final_project


In [1]:
!pip install open-clip-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x8

In [2]:
import os
import torch
import torch.nn as nn
from os.path import expanduser  # pylint: disable=import-outside-toplevel
from urllib.request import urlretrieve  # pylint: disable=import-outside-toplevel
def get_aesthetic_model(clip_model="vit_l_14"):
    """load the aethetic model"""
    home = expanduser("~")
    cache_folder = home + "/.cache/emb_reader"
    path_to_model = cache_folder + "/sa_0_4_"+clip_model+"_linear.pth"
    if not os.path.exists(path_to_model):
        os.makedirs(cache_folder, exist_ok=True)
        url_model = (
            "https://github.com/LAION-AI/aesthetic-predictor/blob/main/sa_0_4_"+clip_model+"_linear.pth?raw=true"
        )
        urlretrieve(url_model, path_to_model)
    if clip_model == "vit_l_14":
        m = nn.Linear(768, 1)
    elif clip_model == "vit_b_32":
        m = nn.Linear(512, 1)
    else:
        raise ValueError()
    s = torch.load(path_to_model)
    m.load_state_dict(s)
    m.eval()
    return m

amodel= get_aesthetic_model(clip_model="vit_l_14")
amodel.eval()

import torch
from PIL import Image
import open_clip
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')

100%|███████████████████████████████████████| 933M/933M [00:11<00:00, 80.3MiB/s]


## Run

In [4]:
import numpy as np

movies = ['frozen', 'spirited_away', 'harry_potter', 'marnie']

In [20]:
all_baselines = []
all_results = []
for movie in movies:
  baseline_scores = np.zeros(3)
  result_scores = np.zeros(3)
  for i in range(3):
    image = preprocess(Image.open(f"{movie}/baseline/output/images/frames/{i * 15}.png")).unsqueeze(0)
    result_image = preprocess(Image.open(f"{movie}/result/output/images/frames/{i * 15}.png")).unsqueeze(0)
    with torch.no_grad():
      image_features = model.encode_image(image)
      image_features /= image_features.norm(dim=-1, keepdim=True)
      prediction = amodel(image_features)
      baseline_scores[i] = prediction

      result_features = model.encode_image(result_image)
      result_features /= result_features.norm(dim=-1, keepdim=True)
      result_prediction = amodel(result_features)
      result_scores[i] = result_prediction
  print(f"Baseline scores for {movie}: {np.mean(baseline_scores)}")
  all_baselines.append(np.mean(baseline_scores))
  print(f"Result scores for {movie}: {np.mean(result_scores)}")
  all_results.append(np.mean(result_scores))
print(f"Average baseline score: {np.mean(all_baselines)}")
print(f"Average result score: {np.mean(all_results)}")

Baseline scores for frozen: 5.9680399894714355
Result scores for frozen: 5.691035906473796
Baseline scores for spirited_away: 6.266530195871989
Result scores for spirited_away: 6.1663510004679365
Baseline scores for harry_potter: 6.090071042378743
Result scores for harry_potter: 6.127197901407878
Baseline scores for marnie: 6.196268876393636
Result scores for marnie: 6.91407585144043
Average baseline score: 6.130227526028951
Average result score: 6.22466516494751
